In [1]:
from hashlib import sha256

def key_gen():
    phi = 2
    e = 2
    while gcd(phi, e) != 1 :
        p = random_prime(2**1024, proof = False)
        q = random_prime(2**1024, proof = False)
        n = p*q
        phi = (p-1) * (q-1)
        e = 65537
    d = inverse_mod(e, phi)
    return (e, d, n, p, q)

def sign(m, d, p, q, n):
    dp = d % (p-1)
    dq = d % (q-1)
    h = int.from_bytes(sha256(m).digest(), byteorder = "big")
    #Nous introduisons ici le bug
    sp = ZZ.random_element(p)# Nous simulons ici le bug. Vrai code: power_mod(h, dp, p)
    sq = power_mod(h, dq, q)
    return crt([sp, sq], [p, q])

def generate():
    (e, d, n, p, q) = key_gen()
    m = b"This message is signed with RSA-CRT!"
    s = sign(m, d, p, q, n)
    print("e = %s" % str(e))
    print("n = %s" %str(n))
    print("s = %s" %s)

generate()

NameError: name 'gcd' is not defined